<a href="https://colab.research.google.com/github/ShoSato-047/DSCI330_Lab-3.1---Text-Classification-with-NLP/blob/main/DSCI330_lab3_1_text_classification_w_NLP_features.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 9.1 MB/s eta 0:00:00


In [2]:
%pip install composable

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 2.7 MB/s eta 0:00:00
  Attempting uninstall: toolz
    Found existing installation: toolz 0.12.1
    Uninstalling toolz-0.12.1:
      Successfully uninstalled toolz-0.12.1


In [4]:
import os
import composable_records as rec
import composable_tuples as tup
from composable import pipeable
from composable.strict import map, filter
from composable_utility import apply, identity, get
from composable_object import obj

# You may need to install utility.py to use "with_open"

In [5]:
# this is the root directory
!pwd

/content


## Loading the emotions dataset

The following data set include a large number of sentences combined with a classification of the emotion.

In [6]:
from datasets import load_dataset

emotions = load_dataset("emotion")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/9.05k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/1.03M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/127k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/129k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/16000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [7]:
emotions

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
})

In [8]:
(train_ds := emotions >> get('train')) #getting training set

Dataset({
    features: ['text', 'label'],
    num_rows: 16000
})

In [9]:
train_ds.column_names

['text', 'label']

In [10]:
train_ds.features

{'text': Value(dtype='string', id=None),
 'label': ClassLabel(names=['sadness', 'joy', 'love', 'anger', 'fear', 'surprise'], id=None)}

In [11]:
train_ds[:5]

{'text': ['i didnt feel humiliated',
  'i can go from feeling so hopeless to so damned hopeful just from being around someone who cares and is awake',
  'im grabbing a minute to post i feel greedy wrong',
  'i am ever feeling nostalgic about the fireplace i will know that it is still on the property',
  'i am feeling grouchy'],
 'label': [0, 0, 3, 2, 3]}

### Preparing the dataset

In [12]:
(documents :=
 train_ds
 >> get('text')
) >> tup.head(5)

['i didnt feel humiliated',
 'i can go from feeling so hopeless to so damned hopeful just from being around someone who cares and is awake',
 'im grabbing a minute to post i feel greedy wrong',
 'i am ever feeling nostalgic about the fireplace i will know that it is still on the property',
 'i am feeling grouchy']

In [13]:
(labels :=
 train_ds
 >> get('label')
) >> tup.head(5)

[0, 0, 3, 2, 3]

### Bag of Words

In [14]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Create the sparse feature set
vectorizer = CountVectorizer()

X = vectorizer.fit_transform(documents)

# Create a training and test (validation) set
X_train, X_test, y_train, y_test = train_test_split(X, labels,
                                                    test_size = 0.2,
                                                    random_state=42)

# Train the model
classifier = MultinomialNB()

classifier.fit(X_train, y_train)

# Evaluate the model
y_pred = classifier.predict(X_test)

(accuracy := accuracy_score(y_test, y_pred))

0.7375

### TF-IDF

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Create the sparse feature set
vectorizer = TfidfVectorizer()

X = vectorizer.fit_transform(documents)

# Create a training and test (validation) set
X_train, X_test, y_train, y_test = train_test_split(X, labels,
                                                    test_size = 0.2,
                                                    random_state=42)

# Train the model
classifier = MultinomialNB()

classifier.fit(X_train, y_train)

# Evaluate the model
y_pred = classifier.predict(X_test)

(accuracy := accuracy_score(y_test, y_pred))

0.6165625

## <font color="red"> Exercise 3.6.1 </font>

The file `train.csv` contains bits of text from three classic authors in the horror genre ([source](https://www.kaggle.com/c/spooky-author-identification/)).

**Preprocessing Tasks.**
1. Read the raw lines in using `with_open`.
2. Split the data into columns and extract the text into one list and the labels into other.
3. Clean up the text by making it lower case and removing any punctuation.
4. Map the text labels to numbers.

**ML tasks.** Test the performance of the naive Bayes classifer on both the Bag of Words and TF-IDF features.

In [ ]:
# Your code here